In [1]:
!pip install itext2kg -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 345.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 580.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 491.4 kB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 519.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 535.5 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: openai
    Found existing installation: openai 1.58.1
    Uninstalling openai-1.58.1:
      Successfully uninstalled openai-1.58.1
ERROR: pip's de

In [ ]:
IE_query = '''
# 指令：
- 像经验丰富的信息提取者一样行动。
- 提取的信息包含人物、地点、事件、物品、任务以及技能。
- 您有大量的故事阅读。
- 如果找不到正确的信息，请将其保留为空白。
'''

In [ ]:
from itext2kg import DocumentDistiller
# You can define a schema or upload some predefined ones.
from itext2kg.utils import Article

# Initialize the DocumentDistiller with llm model.
document_distiller = DocumentDistiller(llm_model = openai_llm_model)

# List of documents to be distilled.
documents = ["doc1", "doc2", "doc3"]



# Distill the documents using the defined query and output data structure.
distilled_doc = document_distiller.distill(documents=documents, IE_query=IE_query, output_data_structure=Article)


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

openai_api_key = "sk-NBj8JhvSyUmrvJMuxUFZd38w858QN0lB2k6voYbxFGlME3Lr"

openai_llm_model = llm = ChatOpenAI(
    api_key = openai_api_key,
    model="claude-3-haiku-20240307",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    base_url="https://chat.cloudapi.vip/v1/"
)

openai_embeddings_model = OpenAIEmbeddings(
    api_key = "sk-1ykrqsQr9OwHePc69A3kbd4JewhK0m8o77Vlp9djVUlmJC9b",
    model="text-embedding-3-small",
    base_url="https://api.chatanywhere.com.cn"
)

In [ ]:
def build_sections(file_path):
    loader = PythonLoader(file_path)
    pages = loader.load_and_split()

    # we have replaced the curly braces with square brackets to avoid the error in the query
    distilled_cv = document_distiller.distill(
        documents=[page.page_content.replace("{", '[').replace("}", "]") for page in pages],
        IE_query=IE_query,
        output_data_structure=CV
    )

    sections = [f"{key} - {value}".replace("{", "[").replace("}", "]") for key, value in distilled_cv.items()
                if value != [] and value != "" and value != None]
    return sections

In [ ]:
def build_graph(sections, existing_global_entities=None, existing_global_relationships=None, ent_threshold=0.6, rel_threshold=0.6):
    global_ent, global_rel = itext2kg.build_graph(
        sections=sections, ent_threshold=ent_threshold, rel_threshold=rel_threshold,
        existing_global_relationships=existing_global_relationships,
        existing_global_entities=existing_global_entities
    )
    print(global_rel)
    print(global_ent)
    return global_ent, global_rel

In [ ]:
sections = build_sections('./datasets/xiyou01.txt')
global_ent, global_rel = build_graph(sections)